In [ ]:
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse, urlunparse
import time

base_url = 'https://www.musashino-u.ac.jp/'
visited_urls = set()
url_title_dict = {}

def normalize_url(u: str) -> str:
    p = urlparse(u)
    
    p = p._replace(fragment='')
    
    p = p._replace(scheme=p.scheme.lower(), netloc=p.netloc.lower())
    return urlunparse(p)


allowed_root = urlparse(base_url).netloc.lower()
def is_same_domain(u: str) -> bool:
    host = urlparse(u).netloc.lower()
    return host == allowed_root or host.endswith('.' + allowed_root)

def scrape_urls(start_url: str):
    stack = [normalize_url(start_url)]
    while stack:
        url = stack.pop()
        if url in visited_urls:
            continue
        visited_urls.add(url)

        try:
            time.sleep(0.5)  
            res = requests.get(url, timeout=10, headers={
                "User-Agent": "MusashinoCrawler/1.0 (+github.com/your-username/musashino-web-crawler)"
            })
        except Exception:
            continue

        ct = res.headers.get('Content-Type', '').lower()
        if not ('text/html' in ct or 'application/xhtml+xml' in ct):
            continue

        
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, 'html.parser')

        
        for comment in soup.find_all(string=lambda t: isinstance(t, Comment)):
            comment.extract()

        
        title = soup.title.string.strip() if soup.title and soup.title.string else 'No Title'
        url_title_dict[url] = title
        print(f"{url} -> {title}")

        
        for a in soup.find_all('a', href=True):
            href = a['href'].strip()
            
            if not href or href.startswith(('#', 'mailto:', 'tel:', 'javascript:')):
                continue
            
            full_url = normalize_url(urljoin(url, href))
            
            if is_same_domain(full_url) and full_url not in visited_urls:
                stack.append(full_url)

if __name__ == "__main__":
    scrape_urls(base_url)
    print("\n=== URLとタイトルの辞書 ===")
    print(url_title_dict)
    print(f"総件数(url_title_dict): {len(url_title_dict)}")
    print(f"総件数(visited_urls): {len(visited_urls)}")

https://www.musashino-u.ac.jp/ -> 武蔵野大学
https://www.musashino-u.ac.jp/accesibility-and-copyright.html -> このサイトについて | 武蔵野大学
https://www.musashino-u.ac.jp/guide/activities/privacypolicy.html -> 個人情報保護方針 | 大学案内 | 武蔵野大学
https://www.musashino-u.ac.jp/business.html -> 企業・研究者の方 | 武蔵野大学
https://www.musashino-u.ac.jp/parents.html -> 保護者の方 | 武蔵野大学
https://www.musashino-u.ac.jp/alumni.html -> 卒業生の方 | 武蔵野大学
https://www.musashino-u.ac.jp/students.html -> 在学生の方 | 武蔵野大学
https://www.musashino-u.ac.jp/prospective-students.html -> 武蔵野大学で学びたい方 | 武蔵野大学
https://www.musashino-u.ac.jp/student-life/ -> 学生生活・就職 | 武蔵野大学
https://www.musashino-u.ac.jp/international/ -> 国際交流・留学 | 武蔵野大学
https://www.musashino-u.ac.jp/research/ -> 研究 | 武蔵野大学
https://www.musashino-u.ac.jp/academics/ -> 学部・大学院 | 武蔵野大学
https://www.musashino-u.ac.jp/academics/basic/ -> 教育の特長 | 武蔵野大学
https://www.musashino-u.ac.jp/admission/ -> 入試情報 | 武蔵野大学
https://www.musashino-u.ac.jp/guide/ -> 大学案内 | 武蔵野大学
https://www.musashino-u.ac.jp/musashino/ -> 交通ア